In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import gridspec
import matplotlib as mpl
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
def load_and_reindex(path,filelist):
    start_time = datetime.now()
    df = None
    for file in filelist:
        year = file[-8:-4]
        manager = file.split('_')[0]
        if df is None:
            df = pd.read_csv(path+file)
            df['year'] = year
            df.index = manager+'_'+year+'_'+df.index.astype(str)
        else:
            temp = pd.read_csv(path+file)
            temp['year'] = year
            temp.index = manager+'_'+year+'_'+temp.index.astype(str)
            df = df.append(temp)
    # adding columns of interest
    df['low_tarif_consumption'] = df['annual_consume'].multiply(df['annual_consume_lowtarif_perc']/100)
    df['num_active_connections'] = df['num_connections'].multiply(df['perc_of_active_connections']/100).astype(int)
    try:
        df['num_smartmeters'] = df['num_connections'].multiply(df['smartmeter_perc']/100).astype(int)
    except ValueError:
        print('Number of smartmeters could not be calculated')
    df['net_annual_consumption'] = df['annual_consume'].multiply(df['delivery_perc']/100)
    df['self_production'] = df['annual_consume'] - df['net_annual_consumption']
    
    time_elapsed = datetime.now() - start_time
    print('Made main dataframe, time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    return(df)

In [ ]:
path = '../input/dutch-energy/Electricity/'
files_all = [f for f in os.listdir(path)]
elec_all = load_and_reindex(path,files_all)
print(elec_all.shape)
print(elec_all.columns)

As a first approach it suffices to look at the energy parameters at the city level. This is convenient, as we then do not have to start messing around with postal code areas (some of which have been created or shuffled around in the intervening years). We can increase the resolution at a later point if that is of interest, but so for now we can make pivot tables containing the aggregate of an energy parameter of interest, per city per year:

In [ ]:
# make pivot tables of relevant parameter such that we have total per city per year
annual_consume = pd.pivot_table(elec_all,values='annual_consume',index='city',columns='year',aggfunc=np.sum)
num_connections = pd.pivot_table(elec_all,values='num_connections',index='city',columns='year',aggfunc=np.sum)
num_active_connections = pd.pivot_table(elec_all,values='num_active_connections',index='city',columns='year',aggfunc=np.sum)
smartmeter_perc = pd.pivot_table(elec_all,values='smartmeter_perc',index='city',columns='year',aggfunc=np.mean)
smartmeter_perc_median = pd.pivot_table(elec_all,values='smartmeter_perc',index='city',columns='year',aggfunc=np.median)
num_smartmeters = pd.pivot_table(elec_all,values='num_smartmeters',index='city',columns='year',aggfunc=np.sum)
self_production = pd.pivot_table(elec_all,values='self_production',index='city',columns='year',aggfunc=np.sum)
net_annu_consume = pd.pivot_table(elec_all,values='net_annual_consumption',index='city',columns='year',aggfunc=np.sum)

In [ ]:
print(annual_consume.sort_values('2018',ascending=False).head())
specs = {'markersize':20,'markerfacecolor':'w','linewidth':2}
plt.plot(annual_consume.columns.astype(int)-1,annual_consume.sum(),'-o',**specs)
#plt.yscale('log')
plt.ylabel('Energy consumption (kWh)')
plt.xlabel('Year')
plt.title('Total yearly energy consumption')

Ok, so the 'year' 2009 - which allegedly covers the consumption in the year before (which is why I changed the x-axis in the plot) - seems to be incomplete. I will therefore drop this year in all following electricity analyses.

First, let's have a look at the country as a whole:

In [ ]:
annual_consume.drop('2009',axis=1,inplace=True)
num_connections.drop('2009',axis=1,inplace=True)
smartmeter_perc.drop('2009',axis=1,inplace=True)
num_smartmeters.drop('2009',axis=1,inplace=True)
self_production.drop('2009',axis=1,inplace=True)
net_annu_consume.drop('2009',axis=1,inplace=True)

In [ ]:
annual_consume.sum()/annual_consume.sum()[0]*100

In [ ]:
f = plt.figure()

gs = gridspec.GridSpec(5,2)
f.set_figwidth(13)
f.set_figheight(10)
plt.suptitle('Electricity Netherlands',fontsize=20)

ax1 = f.add_subplot(gs[0:2,0])
ax1.plot(annual_consume.columns.astype(int)-1,annual_consume.sum(),'-o',**specs)
ax1.plot(annual_consume.columns.astype(int)-1,net_annu_consume.sum(),'-o',**specs)
lgnd = plt.legend(['gross','net'])

#plt.yscale('log')
plt.ylabel('Energy consumption (kWh)')
plt.xlabel('');plt.xticks([])
plt.title('Total yearly energy consumption')

ax11 = f.add_subplot(gs[2,0])
ax11.fill_between(annual_consume.columns.astype(int)-1,0,annual_consume.sum()/annual_consume.sum()[0]*100-100)#,'-o',**specs)
#plt.yscale('log')
plt.ylabel('as % of 2009')
plt.ylim(-4,3)
plt.xlabel('');plt.xticks([])
#plt.title('Total yearly energy consumption')

ax2 = f.add_subplot(gs[0:2,1])
ax2.plot(num_connections.columns.astype(int)-1,num_connections.sum(),'-og',**specs)
#plt.yscale('log')
plt.ylabel('Count')
plt.xlabel('');plt.xticks([])
plt.title('Total number of connections')

ax21 = f.add_subplot(gs[2,1])
ax21.fill_between(annual_consume.columns.astype(int)-1,0,num_connections.sum()/num_connections.sum()[0]*100-100,color='g')#,'-o',**specs)
#plt.yscale('log')
#plt.ylabel('as % of 2008')
plt.xlabel('');plt.xticks([])
#plt.title('Total yearly energy consumption')

ax3 = f.add_subplot(gs[3:,0])
ax3.plot(self_production.columns.astype(int)-1,self_production.sum(),'-oc',**specs)
#plt.yscale('log')
plt.ylabel('Energy  (kWh)')
plt.xlabel('Year')
plt.title('Total yearly self-production')

ax4 = f.add_subplot(gs[3:,1])
ax4.plot(num_smartmeters.columns.astype(int)-1,num_smartmeters.sum(),'-om',**specs)
#plt.yscale('log')
plt.ylabel('Count')
plt.xlabel('Year')
plt.title('Total number of smartmeters')

gs.update(wspace=.51,hspace=.3)





What we see is that the yearly total energy consumption has remained roughly the same, while there is a linear increase in the number of connections. This implies that on average the annual amount of electricity used by a connection (=~household) is going down. In addition, the amount of electricity produced by households has been increasing linearly since 2012. At the same time we see that the number of smartmeters is increasing exponentially and one could expect that the number of smartmeters will start approaching the number of connections in the not too far future. 

In [ ]:
from scipy.optimize import curve_fit
x = num_smartmeters.columns.astype(int)-1
# exponential fit
def exponenial_func(x, a, b, c):
    return 1/(a*np.exp(-b*x)+c)
#p_opt, p_cov = curve_fit(exponenial_func, x, y, p0=(1e-16, 1e-6, ))
xfit = np.linspace(2009,2021,20)
#yfit = exponenial_func(xfit,*p_opt)
# this doesn't seem to work well, since we're working with large numbers here. 
# lin fit in lin-log space does the trick for here (although this is not the most elegant solution):
me,be = np.polyfit(x,np.log(num_smartmeters.sum()),1)
# linear fit
m,b = np.polyfit(x,num_connections.sum(),1)

f,ax = plt.subplots(1,2,figsize=(12,6))

ax[0].plot(num_smartmeters.columns.astype(int)-1,num_connections.sum(),'og',**specs)
ax[0].plot(num_smartmeters.columns.astype(int)-1,num_smartmeters.sum(),'om',**specs)
ax[0].plot(xfit,m*xfit+b,'g')#,'linewidth'=3)
ax[0].plot(xfit,np.exp(me*xfit+be),'m')#,'linewidth'=3)
#plt.yscale('log')
ax[0].set_ylabel('Count')
ax[0].set_xlabel('Year')
ax[0].legend(['# connections','# smartmeters','lin fit','exp fit'],loc=4)
ax[0].set_ylim(bottom=-1e6,top=1e7)
#plt.xlim(0,10)
ax[0].set_title('Smartmeters vs. connections')

specs2 = {'markersize':20,'fillstyle':'none','linewidth':2}
# this is the mean of the mean per city:
ax[1].plot(num_smartmeters.columns.astype(int)-1, smartmeter_perc.mean(),'om',**specs2)
# this is the median of the mean per city:
ax[1].plot(num_smartmeters.columns.astype(int)-1, smartmeter_perc.median(),'og',**specs2)
ax[1].set_title('Smartmeters, percentage of total')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('% of meters being smart')
ax[1].legend(['mean','median'], fontsize=14)

Fitting the data shows that the number of smartmeters should reach the total number of connections in mid 2018. We probably could also have gotten this from the percent of smartmeters parameter in the first place, but let's look at this in greater detail, because simply taking the countrywide mean here oversimplifies what is actually going on. 

For instance, one can notice an increasing discrepancy between the mean and median (of the mean) percentage of smartmeters. This is an indication that we are not dealing with a normal distribution of these things across the cities, which is not a surprise in itself.

In [ ]:
import seaborn as sns

def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

def gauss2(x, *p):
    A1, mu1, sigma1, A2, mu2, sigma2 = p
    return A1*np.exp(-(x-mu1)**2/(2.*sigma1**2)) + A2*np.exp(-(x-mu2)**2/(2.*sigma2**2))

y = '2018'
X_dataG = np.linspace(-1,100,102)
# extract y_data
counts,bins = np.histogram(smartmeter_perc[y],bins=X_dataG,density=True)#[y].hist(bins=40)
y_data = counts
# initial guesses p0
# initialize them differently (one at 20 and the other at 80 %) so the optimization algorithm works better
# this can be circumvented using simulated annealing or sth in the like
p0 = [.03, 20, 1.,.01, 80, 1.]

#optimize and in the end you will have 6 coeff (3 for each gaussian)
coeff, var_matrix = curve_fit(gauss2, X_dataG[1:], y_data, p0=p0)

#plot each gaussian separately 
pg1 = coeff[0:3]
pg2 = coeff[3:]
# using the single gauss function
g1 = gauss(X_dataG, *pg1)
g2 = gauss(X_dataG, *pg2)



f,axs = plt.subplots(3,2,figsize=(12,12))
axs = axs.ravel()
smartmeter_perc['2018'].hist(bins=40,alpha=.3,color='r',ax=axs[0])
#axs[0].set_xlabel('Smartmeters (% of total in a city)')
axs[0].set_ylabel('count')
axs[0].set_title('Distribution of smartmeters per city 2018')
X_data = np.linspace(0,100,40)
elec_all['provider'] = ['liander' if 'liander' in f else 'stedin' if 'stedin' in f else 'enexis' if 'enexis' in f else 'none' for f in elec_all.index]
smart_provider_2018 = pd.pivot_table(elec_all[elec_all.year=='2018'],values='smartmeter_perc',index='city',columns='provider',aggfunc=np.mean)
pleg = []
for provider in smart_provider_2018.columns:
    smart_provider_2018[provider].hist(bins=40,alpha=.3,ax=axs[1])
    pleg.append(provider)
axs[1].legend(pleg)
axs[1].set_title('Smartmeters per city per provider,2018')
leg = []

for i in range(2014,2019):
    y=str(i)
    smartmeter_perc[y].hist(bins=40,alpha=.3,ax=axs[2])
    axs[3].hist(smartmeter_perc[y],bins=X_data,density=True,alpha=.3)
    sns.distplot(smartmeter_perc[y],bins=X_data,hist=False,kde=True,ax=axs[4])
    leg.append(y)
axs[5].hist(smartmeter_perc['2018'],bins=X_data,alpha=.3,color='r',density=True)
axs[5].plot(X_dataG, g1, label='Gaussian1',linewidth=3)
axs[5].plot(X_dataG, g2, label='Gaussian2',linewidth=3)
axs[5].legend()

#ax[2].set_xlabel('Smartmeters (% of total in a city)')
axs[2].set_ylabel('count')
axs[2].set_title('Distribution of smartmeters per city 2014-2018')
axs[2].legend(leg)
axs[3].set_ylabel('Density')
axs[3].set_title('Normalized distribution of smartmeters per city 2014-2018')
axs[4].set_xlabel('Smartmeters (% of total in a city)')
axs[4].legend(leg)
axs[4].set_ylabel('Density')
plt.suptitle('Mean number of smartmeters per city', fontsize=20)


First of all, we see that the distribution is bimodal (2018 shown here): a population of early adopters and a lagging population. It is interesting to look into this - as in: what type of city can be typically found in one or the other - further below. First let's see at what year, given the decline rate of the lagging mode, we expect all cities to have ~80% smartmeters. If we normalize the area under the curve we can fit the data.

Another observation would be that of the three large electricity providers, Enexis is leading the transition to the smart grid.

In [ ]:
gausparams = pd.DataFrame(index=range(2014,2019),columns=['A1','mu1','s1','A2','mu2','s2'])
p0 = [.12, 20, 1.,.01, 80, 1.]
for year in gausparams.index:
    counts,bins = np.histogram(smartmeter_perc[str(year)],bins=X_dataG,density=True)#[y].hist(bins=40)
    y_data = counts
    if year<=2016: #fit with 1 gaussian
        coeff, var_matrix = curve_fit(gauss, X_dataG[1:], y_data, p0=p0[:3])
        gausparams.loc[year,:3] = coeff
    else: #fit with 2
        coeff, var_matrix = curve_fit(gauss2, X_dataG[1:], y_data, p0=p0)
        gausparams.loc[year,:] = coeff

xfit = list(gausparams.index)
y = list(gausparams['A1'])
xpred = np.linspace(2014,2020,20)
m,b = np.polyfit(xfit,y,1)

plt.plot(xfit,y,'og',**specs2,label='Data')
plt.plot(xpred,m*xpred+b,'-g',linewidth=2,label='fit')
plt.axhline(0)
plt.title('Decline of lagging smartmeter population')
plt.xlabel('Year')
plt.ylabel('Density')
plt.legend()

By fitting the "lagging" population with a single gaussian and plotting the (normalized) amplitude vs. year, we see a linear decrease. A simple linear fit shows that this peak is likely to become zero within 2 years of the 2018 timepoint, so around the start of 2020.  So in 2020, one can expect the average city to have around 80% smartmeters. This nicely follows Pareto's law (80% of the work done in 20% of the time and vice versa). If we then assume that the remaining 20% will take 80% of the time, and that the transition started around 2012, the Netherlands will be a 100% smartgrid in 8years/2x8 = 32+2020 = 2052. But this is just a back-of-the-envelope calculation of course. 

In [ ]:
elec_all.columns

In [ ]:
f = plt.figure()
gs = gridspec.GridSpec(1,2)
ax1 = f.add_subplot(gs[0,0])
labels = ['smartmeter_perc','annual_consume_lowtarif_perc','delivery_perc']
dat = elec_all[(elec_all.city.isin(['ROTTERDAM','AMSTERDAM']))&(elec_all.year=='2018')].melt(id_vars='city',value_vars=labels) ##'num_active_connections', 'num_smartmeters'])#
g = sns.violinplot(x='variable',y='value',hue=dat.city,split=True,data=dat,ax=ax1)
plt.xlabel('');plt.ylabel('percentage')
g.set_xticklabels(labels,rotation=30,ha='right')
#plt.yscale('log')
ax2 = f.add_subplot(gs[0,1])
dat = elec_all[(elec_all.city.isin(['UTRECHT',"MAASTRICHT"]))&(elec_all.year=='2018')].melt(id_vars='city',value_vars=labels) ##'num_active_connections', 'num_smartmeters'])#
g = sns.violinplot(x='variable',y='value',hue=dat.city,split=True,data=dat,ax=ax2)
plt.xlabel('');plt.ylabel('')
g.set_xticklabels(labels,rotation=30,ha='right')
f.set_figheight(7)
f.set_figwidth(14)
plt.suptitle('Comparison of individual cities, 2018.')

Let's look at electricity production per city for a bit now.

In [ ]:
f = plt.figure()
gs = gridspec.GridSpec(1,5)

ax1 = f.add_subplot(gs[0,0])
elec_all[elec_all.year=='2018'].groupby('city').sum()['self_production'].divide(1e3).sort_values(ascending=False)[:20].plot.barh(color='darkblue',width=.9,alpha=.7,ax=ax1)
# could have achieved the same with self_production.sort_values('2018',ascending=False).divide(1e3)[:20]
plt.gca().invert_yaxis()
plt.xlabel('Energy produced (MWh)');plt.ylabel('')
plt.title('Self production top 20, 2018')

ax2 = f.add_subplot(gs[0,2])
self_production.divide(num_active_connections).loc[self_production.sort_values('2018',ascending=False).divide(1e3)[:20].index,:].sort_values('2018',ascending=False)['2018'].plot.barh(color='m',width=.9,alpha=.7,ax=ax2)
plt.gca().invert_yaxis()
plt.xlabel('Energy produced per connection (kWh)');plt.ylabel('')
plt.title('Self production per connection, for the top 20 biggest producers, 2018')

ax3 = f.add_subplot(gs[0,4])
self_production.divide(annual_consume/100).loc[self_production.sort_values('2018',ascending=False).divide(1e3)[:20].index,:].sort_values('2018',ascending=False)['2018'].plot.barh(color='g',width=.9,alpha=.7,ax=ax3)
plt.gca().invert_yaxis()
plt.xlabel('Energy produced (% of used)');plt.ylabel('')
plt.title('Self production, % of consumption, top 20 2018')

f.set_figheight(7)
f.set_figwidth(20)



We see that in absolute terms Almere is the biggest producer of energy. When normalized by the number of active connections (which is - I think -  a proxy for the number of inhabitants) this yields the average electricity produced per household per city. The larger cities in the 'randstad' move down the ladder. If we take the actual top 20 cities with the highest average electricity production per household, we expect this ranking to be dominated by small towns in windy (rural) places:

In [ ]:
f = plt.figure()
gs = gridspec.GridSpec(1,3)

ax1 = f.add_subplot(gs[0,0])
self_production.divide(num_active_connections).sort_values('2018',ascending=False)['2018'][:20].plot.barh(color='m',width=.9,alpha=.7,ax=ax1)
plt.gca().invert_yaxis()
plt.xlabel('Energy produced per connection (kWh)');plt.ylabel('')
plt.title('Self production per connection, top 20 2018')

ax2 = f.add_subplot(gs[0,2])
self_production.divide(annual_consume/100).sort_values('2018',ascending=False)['2018'][:20].plot.barh(color='g',width=.9,alpha=.7,ax=ax2)
plt.gca().invert_yaxis()
plt.xlabel('Energy produced (% of consumption)');plt.ylabel('')
plt.title('Self production, % of consumption, top 20 2018')

f.set_figheight(7)
f.set_figwidth(13)

These mostly seem to be rural places in the (north)east. But let's look at the current top 20 and how they ranked over the past years:

In [ ]:
from collections import defaultdict
from scipy import interpolate



def streamgraph(dataframe, **kwargs):
    """ Wrapper around stackplot to make a streamgraph """
    X = dataframe.columns
    Xs = np.linspace(dataframe.columns[0], dataframe.columns[-1], num=1024)
    Ys = [interpolate.PchipInterpolator(X, y)(Xs) for y in dataframe.values]
    return plt.stackplot(Xs, Ys, labels=dataframe.index, **kwargs)

def add_widths(x, y, width=1):
    """ Adds flat parts to widths """
    new_x = []
    new_y = []
    for i,j in zip(x,y):
        new_x += [i-width, i, i+width]
        new_y += [j, j, j]
    return new_x, new_y

def bumpsplot(dataframe, color_dict=defaultdict(lambda: "k"), 
                         linewidth_dict=defaultdict(lambda: 4),
                         labels=[]):
    r = dataframe.rank(method="first")
    r = (r - r.max() + r.max().max()).fillna(0) # Sets NAs to 0 in rank
    for j in r.index:
        x = np.arange(r.shape[1])
        y = r.loc[j].values
        color = color_dict[j]
        lw = linewidth_dict[j]
        x, y = add_widths(x, y, width=0.1)
        xs = np.linspace(0, x[-1], num=1024)
        plt.plot(xs, interpolate.PchipInterpolator(x, y)(xs), color=color, linewidth=lw, alpha=0.5)
        if j in labels:
            plt.text(x[0] - 0.1, y[0], s=j, horizontalalignment="right", verticalalignment="center", color=color)
            plt.text(x[-1] + 0.1, y[-1], s=j, horizontalalignment="left", verticalalignment="center", color=color)
    plt.xticks(np.arange(r.shape[1]), dataframe.columns)
    
    
userank = self_production.drop('2009',axis=1).sort_values('2010',ascending=False)
cities = list(userank[:75].index)
top_cities = list(self_production['2018'].sort_values(ascending=False).index[:20])
finallist = top_cities+list(set(cities).difference(set(top_cities)))
winter_colors = defaultdict(lambda: "grey")
lw = defaultdict(lambda: 1)

top_cities = list(self_production['2018'].sort_values(ascending=False).index[:20])
for i,c in enumerate(top_cities):
    winter_colors[c] = sns.color_palette("husl", n_colors=len(top_cities))[i]
    lw[c] = 4

f = plt.figure(figsize=(18,18))
bumpsplot(userank.loc[finallist,:],color_dict=winter_colors,labels=top_cities)
plt.gca().get_yaxis().set_visible(False)

#plt.gcf().subplots_adjust(left=0.25,bottom=.05,right=.75,top=.95)
sns.despine(left=True)  
plt.title('The road to becoming the top 20 electricity producers 2018')
plt.show()

In [ ]:
self_production.divide(annual_consume/100)['2018'].plot.hist(bins=40)
plt.axvspan(self_production.divide(annual_consume/100)['2018'].mean())

In [ ]:
elec_all['consumption_per_connection'] = elec_all['net_annual_consumption'].divide(elec_all['num_connections'])

In [ ]:
dat = elec_all[(elec_all.city.isin(['UTRECHT',"AMSTERDAM"]))&(elec_all.year=='2018')].melt(id_vars='city',value_vars=['consumption_per_connection']) ##'num_active_connections', 'num_smartmeters'])#
g = sns.violinplot(x='variable',y='value',hue=dat.city,split=True,data=dat)
plt.yscale('log')

In [ ]:
elec_all.groupby('city').mean()['consumption_per_connection'].plot.hist(bins=40)

In [ ]:
"""path = '../input/dutch-energy/Gas/'
files_all = [f for f in os.listdir(path)]
gas_all = load_and_reindex(path,files_all)
print(gas_all.shape)
print(gas_all.columns)"""

In [ ]:
"""f,axs = plt.subplots(3,2,figsize=(12,12))
axs = axs.ravel()
smartmeter_perc_median['2018'].hist(bins=40,alpha=.3,color='r',ax=axs[0])
#axs[0].set_xlabel('Smartmeters (% of total in a city)')
axs[0].set_ylabel('count')
axs[0].set_title('Distribution of smartmeters per city 2018')

elec_all['provider'] = ['liander' if 'liander' in f else 'stedin' if 'stedin' in f else 'enexis' if 'enexis' in f else 'none' for f in elec_all.index]
smart_provider_2018 = pd.pivot_table(elec_all[elec_all.year=='2018'],values='smartmeter_perc',index='city',columns='provider',aggfunc=np.median)
pleg = []
for provider in smart_provider_2018.columns:
    smart_provider_2018[provider].hist(bins=40,alpha=.3,ax=axs[1])
    pleg.append(provider)
axs[1].legend(pleg)
axs[1].set_title('Smartmeters per city per provider,2018')
leg = []
X_data = np.linspace(0,100,40)
for i in range(2014,2019):
    y=str(i)
    smartmeter_perc_median[y].hist(bins=40,alpha=.3,ax=axs[2])
    axs[3].hist(smartmeter_perc_median[y],bins=X_data,density=True,alpha=.3)
    sns.distplot(smartmeter_perc_median[y],bins=X_data,hist=False,kde=True,ax=axs[4])
    leg.append(y)
#ax[2].set_xlabel('Smartmeters (% of total in a city)')
axs[2].set_ylabel('count')
axs[2].set_title('Distribution of smartmeters per city 2014-2018')
axs[2].legend(leg)
axs[3].set_ylabel('Density')
axs[3].set_title('Normalized distribution of smartmeters per city 2014-2018')
axs[4].set_xlabel('Smartmeters (% of total in a city)')
axs[4].legend(leg)
axs[4].set_ylabel('Density')
plt.suptitle('Median number of smartmeters per city', fontsize=20)"""

In [ ]:
f = plt.figure()
gs = gridspec.GridSpec(1,1)
f.set_figheight(8)
f.set_figwidth(8)
ax3 = f.add_subplot(gs[0,0])
ax3.scatter(elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('enexis'))].groupby('city').median()['smartmeter_perc']
         ,elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('enexis'))].groupby('city').median()['annual_consume_lowtarif_perc']
         ,s=elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('enexis'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
ax3.scatter(elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('liander'))].groupby('city').median()['smartmeter_perc']
         ,elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('liander'))].groupby('city').median()['annual_consume_lowtarif_perc']
         ,s=elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('liander'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
ax3.scatter(elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('stedin'))].groupby('city').median()['smartmeter_perc']
         ,elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('stedin'))].groupby('city').median()['annual_consume_lowtarif_perc']
         ,s=elec_all[(elec_all.year=='2018')&(elec_all.index.str.contains('stedin'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
plt.ylabel('mean % of lowtarif consumption')
plt.xlabel('mean % of smartmeters')
plt.title('2018 smartmeter-lowtarif per city\n(sphere radius correlates with city size)')
lgnd = plt.legend(['Enexis','Liander','Stedin'],loc='lower right')
lgnd.legendHandles[0]._sizes = [50]
lgnd.legendHandles[1]._sizes = [50]
lgnd.legendHandles[2]._sizes = [50]

"""ax4 = f.add_subplot(gs[0,0])
ax4.plot(num_smartmeters.columns.astype(int)-1,num_smartmeters.sum(),'-om',**specs)
plt.yscale('log')
plt.ylabel('Count')
plt.xlabel('Year')
plt.title('Total number of smartmeters')
plt.ylim(1e5,1e7)"""

In [ ]:
f, axs = plt.subplots(2,3, figsize=(18, 12))#, facecolor='w', edgecolor='k')
#f.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(2013,2019):
    j=i-2013
    y=str(i)
    axs[j].scatter(elec_all[(elec_all.year==y)&(elec_all.index.str.contains('enexis'))].groupby('city').median()['smartmeter_perc']
             ,elec_all[(elec_all.year==y)&(elec_all.index.str.contains('enexis'))].groupby('city').median()['annual_consume_lowtarif_perc']
             ,s=elec_all[(elec_all.year==y)&(elec_all.index.str.contains('enexis'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
    axs[j].scatter(elec_all[(elec_all.year==y)&(elec_all.index.str.contains('liander'))].groupby('city').median()['smartmeter_perc']
             ,elec_all[(elec_all.year==y)&(elec_all.index.str.contains('liander'))].groupby('city').median()['annual_consume_lowtarif_perc']
             ,s=elec_all[(elec_all.year==y)&(elec_all.index.str.contains('liander'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
    axs[j].scatter(elec_all[(elec_all.year==y)&(elec_all.index.str.contains('stedin'))].groupby('city').median()['smartmeter_perc']
             ,elec_all[(elec_all.year==y)&(elec_all.index.str.contains('stedin'))].groupby('city').median()['annual_consume_lowtarif_perc']
             ,s=elec_all[(elec_all.year==y)&(elec_all.index.str.contains('stedin'))].groupby('city').sum()['num_connections'].divide(1e2),alpha=.2)
    if (i==2013) | (i==2016):
        axs[j].set_ylabel('mean % of lowtarif consumption')
    if i>=2016:
        axs[j].set_xlabel('mean % of smartmeters')
    if i==2018:
        lgnd = plt.legend(['Enexis','Liander','Stedin'],loc='lower right')
        lgnd.legendHandles[0]._sizes = [50]
        lgnd.legendHandles[1]._sizes = [50]
        lgnd.legendHandles[2]._sizes = [50]
plt.suptitle('Evolution of smartmeter-lowtarif per city\n(sphere radius correlates with city size)',fontsize=25)


In [ ]:
elec_all[elec_all.year=='2018'].type_of_connection.value_counts().sort_values().plot.barh(color='b',alpha=.3,width=.9,figsize=(7,7))
plt.xscale('log')


In [ ]:
sns.distplot(smartmeter_perc[y],bins=X_data,hist=True,kde=True)

In [ ]:
counts


In [ ]:
bins

In [ ]:
num_smartmeters['2018'].hist(bins=np.logspace(1,5,20))
plt.xscale('log')

In [ ]:
num_connections['2018'].hist(bins=np.logspace(1,5,20),alpha=.3)
plt.xscale('log')

In [ ]:
#plt.style.use(['fivethirtyeight'])
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
plt.plot(annual_consume.columns.astype(int)-1,self_production.sum()/annual_consume.sum()*100,'-o',**specs)
#plt.yscale('log')
plt.ylabel('% of total consumption')
plt.xlabel('');#plt.xticks([])
plt.title('Self-produced energy in NL')

In [ ]:
set(elec_enexis.city.unique()).intersection(set(elec_stedin.city.unique()))

In [ ]:
len(elec_liander.city.unique())
elec_liander.shape

In [ ]:
len(elec_stedin.city.unique())

QUESTIONS global
when will the number of smartmeters equal the number of connections?
when will the amount of self-produced energy become a significant fraction (20,30,50%)? What could limit growth here?
How are the energy providers performing, is any company increasing its market share? Who is leading the smartmeter business?
Will the total consumption go up or down in the future?
How does this compare to gas consumption?
Can we detect difference in consumption of gas depending on the average monthly/yearly temperature?

Then continue on a city level:
rankings, bump plots 
city growth rates
are there places stagnating and/or left behind?



In [ ]:
def get_yearly_percity(df):
    years = df['year'].unique()
    group = None
    keys = ['num_connections','annual_consume']
    for y in years:
        temp = df[df['year']==y]
        temp_group = temp.groupby('city').sum()[keys]
        values = [keys[0]+'_'+y,keys[1]+'_'+y]
        if group is None:
            group = temp_group.rename(index=str,columns=dict(zip(keys,values)))
        else:
            temp_group.rename(index=str,columns=dict(zip(keys,values)),inplace=True)
            group = group.join(temp_group)
    return(group)

def plot_cities_yearly(groupdf,years):
    yearsint = [int(f) for f in years]
    

In [ ]:
all_years_cities = get_yearly_percity(elec_all)

In [ ]:
all_years_cities.sort_values('num_connections_2018',ascending=False)[0:20]

In [ ]:
#elec_all.pivot(index='city',columns='year',values='annual_consume')#['variable'].unique()
#elec_all.city.value_counts()
annual_consume = pd.pivot_table(elec_all,values='annual_consume',index='city',columns='year',aggfunc=np.sum)
num_connections = pd.pivot_table(elec_all,values='num_connections',index='city',columns='year',aggfunc=np.sum)
smartmeter_perc = pd.pivot_table(elec_all,values='smartmeter_perc',index='city',columns='year',aggfunc=np.mean)

In [ ]:
smartmeter_perc.sort_values('2018',ascending=False)

In [ ]:
top20_connect = num_connections.sort_values('2018',ascending=False)[0:20]
top20_con_consume = annual_consume.loc[top20_connect.index,:]
years = [int(f) for f in top20_connect.columns]
"""f = plt.figure()
gs = gridspec.GridSpec(1,1)

ax1 = f.add_subplot(gs[0,0])
for city in top20_connect.index:
    """


In [ ]:
ax = top20_con_consume.drop('EINDHOVEN').T.plot()
#plt.yscale('lin')


In [ ]:
plt.scatter(top20_connect.loc[:,'2018'],top20_con_consume.loc[:,'2018'])


In [ ]:
plt.loglog(num_connections.loc[:,'2018'],annual_consume.loc[:,'2018'],'o',alpha=.4)



In [ ]:
top

In [ ]:
f, ax = plt.subplots(figsize = (10,15))
m = Basemap(resolution='h', # c, l, i, h, f or None
            projection='merc',
            lat_0=54.5, lon_0=-4.36,
            llcrnrlon=3.15, llcrnrlat= 50.7, urcrnrlon=7.3, urcrnrlat=53.84)
m.drawmapboundary(fill_color='#46bcec')
m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')
m.drawcoastlines()

In [ ]:
path = '../input/dutch-energy/Electricity/'
listoffiles = [f for f in os.listdir(path)]
print(listoffiles)
elec_data = load_and_reindex(path,listoffiles)

In [ ]:
elec_data.columns

In [ ]:
elec_data.groupby('year').sum()['annual_consume'].plot.bar(color='blue',alpha=.4)

In [ ]:
elec_data_2009 = elec_data[elec_data['year']=='2009']
elec_data_2010 = elec_data[elec_data['year']=='2010']

In [ ]:
elec_data_2009.index.str[0:6].unique()
elec_data_2010.index.str[0:6].unique()#.groupby('city').sum()

In [ ]:
elec_data_2010